In [1]:
import matplotlib.pyplot as plt
import pickle
import numpy as np

In [12]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [18]:
record = pickle.load(open('air_flip_record.pkl', 'rb'))
play = pickle.load(open('air_flip_play.pkl', 'rb'))
rate = 48000

x_axis = np.arange(0, len(record)) / rate
# plt.plot(x_axis, play)
# plt.plot(x_axis, record)
# plt.show()
# plt.xlim(0, 0.025)

fig, ax = plt.subplots()
ax.plot(x_axis, play, label='Playback')
ax.plot(x_axis, record, label='Capture')
ax.set_xlim(-0.0002, 0.005)
# show label
ax.legend()
# set label
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
# set title
ax.set_title('Phase Flip')

fig.set_size_inches(w=4.7747 * 1.5, h=2)
plt.savefig('../report/air_flip.pgf')

In [19]:
record = pickle.load(open('air_drift_record.pkl', 'rb'))
play = pickle.load(open('air_drift_play.pkl', 'rb'))
rate = 48000

x_axis = np.arange(0, len(record)) / rate
# plt.plot(x_axis, play)
# plt.plot(x_axis, record)
# plt.show()
# plt.xlim(0, 0.025)

fig, ax = plt.subplots()
ax.plot(x_axis, play, label='Playback')
ax.plot(x_axis, record, label='Capture')
ax.set_xlim(0.0044, 0.01055)
# show label
ax.legend()
# set label
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
# set title
ax.set_title('Clock Drift of Sine Wave (2000 Hz)')

cycle_t = 1 / 2000

i = 0
while i * cycle_t < 0.02:
    ax.axvline(i * cycle_t, color='gray', linestyle='--')
    i += 1

    start = i * 24
    end = start + 24

    signal = play[start:end]
    cap_signal = record[start:end]

    cap_signal = cap_signal / np.max(np.abs(cap_signal))

    dot_product = np.dot(signal, cap_signal) / 24 * 1

    # draw the dot product
    ax.text(i * cycle_t + cycle_t / 8, 0.25, f'{dot_product:.2f}', fontsize=8, color='red', clip_on=True)


fig.set_size_inches(w=4.7747 * 1.5, h=2)
plt.savefig('../report/air_drift.pgf')

In [20]:
record = pickle.load(open('line_delay_record.pkl', 'rb'))
play = pickle.load(open('line_delay_play.pkl', 'rb'))

rate = 48000
x_range = np.arange(0, len(record)) / rate

fig, ax = plt.subplots()

ax.plot(x_range, play, label='Playback')
ax.plot(x_range, record, label='Capture')
ax.set_xlim(0.098, 0.19)
# show label
ax.legend()
# set label
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
# set title
ax.set_title('Latency')

# draw the vertical line at x=0.18
ax.axvline(0.18, color='gray', linestyle='--')
ax.axvline(0.1, color='gray', linestyle='--')


fig.set_size_inches(w=4.7747 * 1.5, h=2)
plt.savefig('../report/line_latency.pgf')

In [15]:
# chirp

def gen_chirp(f_0, c, duration, rate):
    f_1 = c * duration + f_0
    print(f"chrip freq {f_0}, {f_1}")

    def phi(t):
        return 2 * np.pi * (c / 2 * t * t + f_0 * t)

    times = np.linspace(0, duration, int(rate * duration)+1)[:-1]
    chirp = np.sin(phi(times))
    chirp_rev = -np.flip(chirp)
    chirp = np.concatenate((chirp, chirp_rev))
    # chirp = np.concatenate((chirp, np.zeros(int(rate * duration))))
    return chirp

rate = 48000
chirp_fine = gen_chirp(5000, 5000000, 0.001, rate * 4)

# plt.plot(chirp_fine)
fig, ax = plt.subplots()
x_axis = np.arange(0, len(chirp_fine)) / rate
ax.plot(x_axis, chirp_fine)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('Chirp Signal')

fig.set_size_inches(w=4.7747 * 1.5, h=2)
plt.savefig('../report/chirp.pgf')

chrip freq 5000, 10000.0


In [16]:
record = pickle.load(open('line_coding_record.pkl', 'rb'))
signal = pickle.load(open('line_coding_signal.pkl', 'rb'))

# plt.plot(record)
# plt.plot(signal)

# instead of line, we plot dots
# plt.plot(record, '-o')
# plt.plot(signal, '-o')

fig, ax = plt.subplots()

for i in range(len(signal) // 2):
    ax.plot([i*2, i*2+1], [signal[i*2], signal[i*2+1]], '-o', color='C0')
    ax.plot([i*2, i*2+1], [record[i*2], record[i*2+1]], '-o', color='C1')
    ax.axvline(i*2+1.5, color='gray', linestyle='--')

ax.legend(['Playback', 'Capture'])

ax.set_xlabel('Samples')
ax.set_ylabel('Amplitude')
ax.set_title('ASK on Cable')

fig.set_size_inches(w=4.7747 * 1.5, h=2)
plt.savefig('../report/ask.pgf')


# plt.plot(signal)
# plt.plot(record)
# plt.show()

In [18]:
play = pickle.load(open('fsk_play.pkl', 'rb'))
record = pickle.load(open('fsk_record.pkl', 'rb'))
fig, ax = plt.subplots()
rate = 48000
x_axis = np.arange(0, len(record)) / rate
ax.plot(x_axis, play, label='Playback')
ax.plot(x_axis, record, label='Capture')
ax.legend()
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('FSK via Air (1000Hz and 2000Hz)')
freq = 1000
symbol_samples = int(rate / freq)
for i in range(1, 5):
    ax.axvline(x=(i*symbol_samples/rate), color='grey', linestyle='--')

fig.set_size_inches(w=4.7747 * 1.5, h=2)
plt.savefig('../report/fsk.pgf')